<a href="https://colab.research.google.com/github/benjamindeluca/Capstone/blob/main/Copy_of_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

import tensorflow

from matplotlib import pyplot as plt


# Training will be significantly faster if GPU is available. In Colab, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")
# set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


GPU is not available


Import data:
Download the train.zip file from https://www.kaggle.com/c/grasp-and-lift-eeg-detection/data and
explore the dataset. This file contains the first 8 series for each subject. (We will be only using
train.zip for the project.)
There are two files for each subject + series combination:
● the *_data.csv files contain the raw 32 channels EEG data (sampling rate 500Hz)
● the *_events.csv files contains the ground truth frame-wise labels for all events

NOTE: to import the data you have to log into kaggle and create an API token. From there you upload the kaggle.json file that will be downloaded when creating the API token and upload it into the connected google drive account for colab. Then give colab permission to access the files when running the below lines.

In [3]:
! pip install kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [7]:
! kaggle competitions download grasp-and-lift-eeg-detection --force

 99% 1.01G/1.02G [00:11<00:00, 71.0MB/s]
100% 1.02G/1.02G [00:12<00:00, 91.0MB/s]


In [8]:
import zipfile
import pandas as pd
import os

# Open the zip file
with zipfile.ZipFile('grasp-and-lift-eeg-detection.zip', 'r') as zip_ref:
    # Extract all files
    zip_ref.extractall('grasp-and-lift-eeg-detection')

# List the extracted files
extracted_files = zip_ref.namelist()

display(extracted_files)

['sample_submission.csv.zip', 'test.zip', 'train.zip']

In [9]:
# Function to read data and labels
def read_data_and_labels(zip_file):
    train_data = {}
    test_data = {}
    test_labels = {}
    train_labels = {}

    # Extracted folder name will be the same as the zip file name without the extension
    extracted_folder = os.path.splitext(zip_file)[0]

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all files
        zip_ref.extractall(extracted_folder)

    extracted_files = zip_ref.namelist()

    for file_name in extracted_files:
        if file_name.endswith('_data.csv'):
            subject_id, series = file_name.split('_')[:2]
            df = pd.read_csv(os.path.join(extracted_folder, file_name))
            if series == 'series7' or series == 'series8':
                test_data.setdefault(subject_id, []).append(df)
            else:
                train_data.setdefault(subject_id, []).append(df)

        elif file_name.endswith('_events.csv'):
            subject_id, series = file_name.split('_')[:2]
            labels_df = pd.read_csv(os.path.join(extracted_folder, file_name))

            if series == 'series7' or series == 'series8':
              test_labels.setdefault(subject_id, []).append(labels_df)
            else:
              train_labels.setdefault(subject_id, []).append(labels_df)


    return train_data, test_data, test_labels, train_labels

# Read data and labels
train_data, test_data, test_labels, train_labels = read_data_and_labels('grasp-and-lift-eeg-detection/train.zip')



In [10]:
print(train_data.keys())
print(test_data.keys())

# Example: Accessing data for subject '1'
subject_id = 'train/subj1'
train_data_subject_1 = pd.concat(train_data[subject_id], ignore_index=True)
test_data_subject_1 = pd.concat(test_data[subject_id], ignore_index=True)
test_labels_subject_1 = pd.concat(test_labels[subject_id], ignore_index=True)
train_labels_subject_1 = pd.concat(train_labels[subject_id], ignore_index=True)

# display(test_labels_subject_1)
# Now you have train and test data along with labels for each subject

dict_keys(['train/subj10', 'train/subj11', 'train/subj12', 'train/subj1', 'train/subj2', 'train/subj3', 'train/subj4', 'train/subj5', 'train/subj6', 'train/subj7', 'train/subj8', 'train/subj9'])
dict_keys(['train/subj10', 'train/subj11', 'train/subj12', 'train/subj1', 'train/subj2', 'train/subj3', 'train/subj4', 'train/subj5', 'train/subj6', 'train/subj7', 'train/subj8', 'train/subj9'])


# Preprocess the data
Explore on EEG signal preprocessing methods and make choices on what sort of preprocessing this
dataset would need. You can justify your choices in the report

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

# Custom Dataset Class
class EEGDataset(Dataset):
    def __init__(self, data_dict_raw, labels_dict):
        self.data_dict = data_dict_raw
        self.labels_dict = labels_dict

        # Extract subject IDs and series
        self.subject_names = list(data_dict_raw.keys())
        self.series = ['series1', 'series2', 'series3', 'series4', 'series5', 'series6','series7', 'series8', 'series9', 'series10', 'series11', 'series12']

        # broken
        self.subjects = {}
        for subject in self.subject_names:
          self.subjects.update({str(subject): data_dict_raw[subject]})

    def __len__(self):
        return len(self.subject_names) * len(self.series)

    def __getitem__(self, idx):
        subject_idx = idx // len(self.series)
        series_idx = idx % len(self.series)
        subject_id = self.subject_names[subject_idx]
        series = self.series[series_idx]

        # Get EEG data and labels for the current subject and series
        eeg_data = self.subjects[subject_id][series_idx]
        labels = self.labels_dict[subject_id]

        return eeg_data.values, labels.values

    # Need a function to update the data_dict


# Create Dataset instances
# print(train_data['train/subj10'][1])
train_dataset = EEGDataset(train_data, train_labels)
test_dataset = EEGDataset(test_data, test_labels)

print(train_dataset.subjects['train/subj10'][0])

# print(train_dataset.data_dict)

                           id        Fp1        Fp2          F7         F3  \
0            subj10_series1_0  -4.497380 -29.211153  -59.151118 -26.991991   
1            subj10_series1_1  10.257334 -58.665263  -38.470355 -64.551683   
2            subj10_series1_2  16.140461 -84.515489  -41.236517 -90.158499   
3            subj10_series1_3  12.358437 -90.343431  -63.392971 -97.427322   
4            subj10_series1_4   5.988439 -63.986183  -74.776784 -87.126075   
...                       ...        ...        ...         ...        ...   
262382  subj10_series1_262382  28.341714 -39.437041  -14.427492  92.584966   
262383  subj10_series1_262383  46.953302  -9.218532   49.531051  86.017622   
262384  subj10_series1_262384  61.373963  25.123945   96.284843  74.320547   
262385  subj10_series1_262385  59.464584  33.134911  100.920553  56.044810   
262386  subj10_series1_262386  41.813923  11.084058   66.367898  31.829118   

               Fz         F4         F8         FC5        FC1 

Filtering the Data


In [16]:
import matplotlib.pyplot as plt
from scipy import signal

# Making a copy of the dataset so as the maintain the original
filtered_train = train_dataset
filtered_test = test_dataset

def filter_eeg_dataset(EEGDataset, f_low, f_high, order, type, fs):
  b, a = signal.butter(order, [f_low, f_high], btype=type, fs=fs)

  subject_names = EEGDataset.subject_names

  # For each Subject in EEG Dataset
  for subject in subject_names:
    # For each Series in Subject
    for series in range(6):
      try:
        data_trial = EEGDataset.subjects[subject][series]
        # For each Column in series and subject
        # Not the first non-numerical columns
        for col_name in data_trial.columns:

          if col_name != "id":
            filtered_data = signal.filtfilt(b, a, data_trial[col_name])

            # Replace data
            EEGDataset.subjects[subject][series][col_name] = filtered_data


      # Breaking when we run out of series
      except IndexError as e:
        print(str(subject) + " caps at "+ str(series))
        break




# System Params
order = 4
sampling_frequency = 500

# High and Low Pass Filter
f_low = 1
f_high = 100
f_type = 'bandpass'

filter_eeg_dataset(filtered_train, f_low, f_high, order, f_type, sampling_frequency)
filter_eeg_dataset(filtered_test, f_low, f_high, order, f_type, sampling_frequency)

# Bandreject
f_low = 49
f_high = 51
f_type = 'bandstop'

filter_eeg_dataset(filtered_train, f_low, f_high, order, f_type, sampling_frequency)
filter_eeg_dataset(filtered_test, f_low, f_high, order, f_type, sampling_frequency)

train/subj10 caps at 2
train/subj11 caps at 2
train/subj12 caps at 2
train/subj1 caps at 2
train/subj2 caps at 2
train/subj3 caps at 2
train/subj4 caps at 2
train/subj5 caps at 2
train/subj6 caps at 2
train/subj7 caps at 2
train/subj8 caps at 2
train/subj9 caps at 2
train/subj10 caps at 2
train/subj11 caps at 2
train/subj12 caps at 2
train/subj1 caps at 2
train/subj2 caps at 2
train/subj3 caps at 2
train/subj4 caps at 2
train/subj5 caps at 2
train/subj6 caps at 2
train/subj7 caps at 2
train/subj8 caps at 2
train/subj9 caps at 2


Feature Extraction


# Define Neural Network

In [17]:
# Create DataLoader instances
batch_size = 32
train_loader = DataLoader(filtered_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(filtered_test, batch_size=batch_size, shuffle=False)

class EEGModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(EEGModel, self).__init__()
        # Define your neural network architecture here
        # Example:
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.fc3 = nn.Conv1d()

    def forward(self, x):
        # Define the forward pass of your network
        # Example:
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Conv1DModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size):
        super(Conv1DModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=num_channels, kernel_size=kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Max pooling layer
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels*2, kernel_size=kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Max pooling layer
            nn.Conv1d(in_channels=num_channels*2, out_channels=num_channels*4, kernel_size=kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)   # Max pooling layer
        )
        # Calculate output size after convolutions
        self.fc_input_size = num_channels*4 * ((input_size - 2*(kernel_size-1)) // 2**3)
        # Fully connected layer
        self.fc_layers = nn.Sequential(
            nn.Linear(self.fc_input_size, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, self.fc_input_size)  # Reshape for fully connected layers
        x = self.fc_layers(x)
        return x


# We could make a bi-directional RNN, which violates real-time detection, but can analyse other recorded test sets

In [ ]:
! pip install nnspt

In [19]:
# Get the input size from the train_loader
# for inputs, _ in train_loader:
#     input_size = inputs.shape[1]  # Get the number of features from the shape of the input data
#     break  # Break after extracting the first batch
from sklearn import metrics


# Define the neural network model with the correct input size
hidden_size = 64  # Example hidden size
num_classes = 6  # Example number of classes

# model = Conv1DModel(input_size, hidden_size, num_classes)
epochs = 20

model = Conv1DModel(input_size=32, output_size=6, num_channels=64, kernel_size = 200)
model.to(device)
optimiser = torch.optim.AdamW(model.parameters(), lr=0.001)
sheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, epochs*len(train_dataloader))



ModuleNotFoundError: No module named 'nnspt'